<a href="https://colab.research.google.com/github/shirleyrutgers/DataVis/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PS1

1.  Explain why you use datasets youre using: abstract with res q and hypotheses; extend and improve as we go over time the writeup must be longer and include at least: why these data, whats the research questions, hypotheses and specific variables you are interested in and why?

### **Mini Abstract - Research Context and Data Justification**  

#### **Research Topic:**  
**The Relationship Between County GDP and Business Formation in New Jersey**  

#### **Research Question:**  
**Do counties with higher GDP levels have a greater number of business applications in New Jersey?**  

#### **Hypothesis:**  
Counties in New Jersey with higher GDP levels will exhibit a higher number of new business applications, as economic growth fosters entrepreneurship through increased access to capital, consumer demand, and business opportunities.  

#### **Why These Data?**  
To explore this relationship, we use three datasets:  
1. **bfs_county_apps_annual.xlsx** – Contains annual business application counts by county, allowing us to measure entrepreneurship levels.  
2. **NJCountyData.xlsx** – Provides GDP per county, essential for understanding economic strength.  
3. **NJwage.xlsx** – Includes wage data by county, which helps control for income effects on entrepreneurship.  

#### **Key Variables of Interest:**  
- **Business Applications (Dependent Variable):** Measures entrepreneurship activity.  
- **GDP per County (Independent Variable):** Represents economic development and financial capacity.  
- **Median Income & Wages (Control Variables):** Adjust for economic disparities affecting entrepreneurship.  

This study will contribute to understanding how regional economic conditions impact business formation, providing insights for policymakers and economic development strategies.

Data
NJCountyData (2025)> https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.njeda.gov%2Fwp-content%2Fuploads%2F2021%2F05%2FNJCountyData-1.xlsx&wdOrigin=BROWSELINK

NJ data, population, gdp
https://docs.google.com/spreadsheets/d/e/2PACX-1vRCnEjufrgpP0UB0Quxx8nYC56Yw0C7ZuWMDjPj9H8BrUF8hPGSPBpmcGlJKc0xhA/pub?gid=2071317397&single=true&output=csv


Business Formation Statistics - Annual County Data
Business Applications by County (2025)

https://www.census.gov/econ/bfs/data/county.html

NJ business applications https://docs.google.com/spreadsheets/d/e/2PACX-1vSzhIWXDHIusT7myBctQLeKRlPvg7V7YVIweyIMU7W3I3NKX2UjTMNKPA995O1JTQ/pub?gid=1018220772&single=true&output=csv

NJ wage
https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.nj.gov%2Flabor%2Flabormarketinformation%2Fassets%2FPDFs%2Femploy%2Fqcew%2Fcvremp23.xlsx&wdOrigin=BROWSELINK

https://docs.google.com/spreadsheets/d/e/2PACX-1vS8byZhwUGPcnozsRjz2WuI2lUIi84AweOFjg1lM0lSLpEzRQhK-NNBZdKZK9XoEw/pub?gid=69402389&single=true&output=csv




2.  Use at least 2 times each of the following in a way that makes sense and is useful:

 • rename var:
 • replace vals
 • drop or keep vars
 • collabse data (groupby agg)

In [ ]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

Step 1. Download file from online and save in colab

Data

NJ data, population, gdp
https://docs.google.com/spreadsheets/d/e/2PACX-1vRCnEjufrgpP0UB0Quxx8nYC56Yw0C7ZuWMDjPj9H8BrUF8hPGSPBpmcGlJKc0xhA/pub?gid=2071317397&single=true&output=csv


NJ business applications https://docs.google.com/spreadsheets/d/e/2PACX-1vSzhIWXDHIusT7myBctQLeKRlPvg7V7YVIweyIMU7W3I3NKX2UjTMNKPA995O1JTQ/pub?gid=1018220772&single=true&output=csv

NJ wage
https://docs.google.com/spreadsheets/d/e/2PACX-1vS8byZhwUGPcnozsRjz2WuI2lUIi84AweOFjg1lM0lSLpEzRQhK-NNBZdKZK9XoEw/pub?gid=69402389&single=true&output=csv

Sources:

Youtube tutorial. "Como importar CSV a Google Colab", (2025). https://www.youtube.com/watch?v=KCbpPhr_7DY

Youtube tutorial. "Dos alternativas para cargar un archivo CSV a nuestro colab", (2025). https://www.youtube.com/watch?v=pLAoxHXOeuk

In [ ]:
import pandas as pd
df1 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRCnEjufrgpP0UB0Quxx8nYC56Yw0C7ZuWMDjPj9H8BrUF8hPGSPBpmcGlJKc0xhA/pub?gid=2071317397&single=true&output=csv')

In [ ]:
df2=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSzhIWXDHIusT7myBctQLeKRlPvg7V7YVIweyIMU7W3I3NKX2UjTMNKPA995O1JTQ/pub?gid=1018220772&single=true&output=csv')

In [ ]:
df3=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS8byZhwUGPcnozsRjz2WuI2lUIi84AweOFjg1lM0lSLpEzRQhK-NNBZdKZK9XoEw/pub?gid=69402389&single=true&output=csv')

In [ ]:
df1.head()
df2.head()
df3.head()

,County,Employment,Unemployment,Population,GDP,Median Income
0,Atlantic,100709,12.5,"266,105",12593490,62678
1,Bergen,426341,7.6,"930,390",70158193,107971
2,Burlington,210681,6.6,"445,702",25725252,88443
3,Camden,229229,8.2,"506,738",23558717,73168
4,Cape May,39046,12.6,"93,086",5070716,66565


,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,...,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022,BA2023
0,NJ,Atlantic County,34001,34,1,2281,2314,2336,2227,2050,...,2163,2161,2113,2195,2275,2501,3295,4082,3862,3885
1,NJ,Bergen County,34003,34,3,11218,11633,11833,11537,11091,...,11854,12345,12533,13497,14780,14225,15508,18500,17576,18508
2,NJ,Burlington County,34005,34,5,3816,3904,4103,3927,3470,...,3576,3601,3704,4084,4534,4732,5915,7617,6977,7703
3,NJ,Camden County,34007,34,7,3953,4322,4398,4336,3812,...,3855,3918,4107,4321,4740,5131,7073,8994,8030,8225
4,NJ,Cape May County,34009,34,9,1080,1002,962,915,774,...,803,1014,833,863,906,925,1052,1334,1284,1311


,County,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Atlantic,"$ 37,387","$ 38,568","$ 39,665","$ 40,362","$ 41,682","$ 42,850","$ 43,772","$ 49,306","$ 51,055","$ 52,852","$ 54,201"
1,Bergen,"$ 60,510","$ 62,191","$ 63,085","$ 63,323","$ 63,997","$ 65,378","$ 66,972","$ 73,426","$ 75,903","$ 77,278","$ 79,275"
2,Burlington,"$ 50,801","$ 51,597","$ 53,798","$ 53,456","$ 54,639","$ 56,829","$ 58,262","$ 64,983","$ 66,584","$ 67,974","$ 68,992"
3,Camden,"$ 47,212","$ 46,680","$ 49,078","$ 49,585","$ 51,162","$ 52,445","$ 54,335","$ 59,994","$ 62,506","$ 63,260","$ 64,846"
4,Cape May,"$ 30,578","$ 31,183","$ 32,361","$ 33,248","$ 33,359","$ 34,088","$ 35,378","$ 39,104","$ 41,103","$ 42,920","$ 44,209"


Rename

In [ ]:
df1.rename(columns={"GDP": "Gross Domestic Product"}, inplace=True)

In [ ]:
df1.head()

,County,Employment,Unemployment,Population,Gross Domestic Product,Median Income
0,Atlantic,100709,12.5,"266,105",12593490,62678
1,Bergen,426341,7.6,"930,390",70158193,107971
2,Burlington,210681,6.6,"445,702",25725252,88443
3,Camden,229229,8.2,"506,738",23558717,73168
4,Cape May,39046,12.6,"93,086",5070716,66565


In [ ]:
df3.rename(columns={
    "2013": "Wage 2013",
    "2014": "Wage 2014",
    "2015": "Wage 2015",
    "2016": "Wage 2016",
    "2017": "Wage 2017",
    "2018": "Wage 2018",
    "2019": "Wage 2019",
    "2020": "Wage 2020",
    "2021": "Wage 2021",
    "2022": "Wage 2022",
    "2023": "Wage 2023"
}, inplace=True)

In [ ]:
df3.head()

,County,Wage 2013,Wage 2014,Wage 2015,Wage 2016,Wage 2017,Wage 2018,Wage 2019,Wage 2020,Wage 2021,Wage 2022,Wage 2023
0,Atlantic,"$ 37,387","$ 38,568","$ 39,665","$ 40,362","$ 41,682","$ 42,850","$ 43,772","$ 49,306","$ 51,055","$ 52,852","$ 54,201"
1,Bergen,"$ 60,510","$ 62,191","$ 63,085","$ 63,323","$ 63,997","$ 65,378","$ 66,972","$ 73,426","$ 75,903","$ 77,278","$ 79,275"
2,Burlington,"$ 50,801","$ 51,597","$ 53,798","$ 53,456","$ 54,639","$ 56,829","$ 58,262","$ 64,983","$ 66,584","$ 67,974","$ 68,992"
3,Camden,"$ 47,212","$ 46,680","$ 49,078","$ 49,585","$ 51,162","$ 52,445","$ 54,335","$ 59,994","$ 62,506","$ 63,260","$ 64,846"
4,Cape May,"$ 30,578","$ 31,183","$ 32,361","$ 33,248","$ 33,359","$ 34,088","$ 35,378","$ 39,104","$ 41,103","$ 42,920","$ 44,209"


Replace

In [39]:
# Replace county names by removing "County"
df2["County"] = df2["County"].replace({
    "Atlantic County": "Atlantic",
    "Bergen County": "Bergen",
    "Burlington County": "Burlington",
    "Camden County": "Camden",
    "Cape May County": "Cape May",
    "Cumberland County": "Cumberland",
    "Essex County": "Essex",
    "Gloucester County": "Gloucester",
    "Hudson County": "Hudson",
    "Hunterdon County": "Hunterdon",
    "Mercer County": "Mercer",
    "Middlesex County": "Middlesex",
    "Monmouth County": "Monmouth",
    "Morris County": "Morris",
    "Ocean County": "Ocean",
    "Passaic County": "Passaic",
    "Salem County": "Salem",
    "Somerset County": "Somerset",
    "Sussex County": "Sussex",
    "Union County": "Union",
    "Warren County": "Warren"
})

# Verify the changes
print(df2.head())


  State      County  County Code  fipstate  fipscty  BA2005  BA2006  BA2007  \
0    NJ    Atlantic        34001        34        1    2281    2314    2336   
1    NJ      Bergen        34003        34        3   11218   11633   11833   
2    NJ  Burlington        34005        34        5    3816    3904    4103   
3    NJ      Camden        34007        34        7    3953    4322    4398   
4    NJ    Cape May        34009        34        9    1080    1002     962   

   BA2008  BA2009  ...  BA2014  BA2015  BA2016  BA2017  BA2018  BA2019  \
0    2227    2050  ...    2163    2161    2113    2195    2275    2501   
1   11537   11091  ...   11854   12345   12533   13497   14780   14225   
2    3927    3470  ...    3576    3601    3704    4084    4534    4732   
3    4336    3812  ...    3855    3918    4107    4321    4740    5131   
4     915     774  ...     803    1014     833     863     906     925   

   BA2020  BA2021  BA2022  BA2023  
0    3295    4082    3862    3885  
1   1550

In [40]:
# Replace "NJ" with "New Jersey" in the "State" column
df2["State"] = df2["State"].replace("NJ", "New Jersey")

# Display the first few rows to verify changes
print(df2.head())

        State      County  County Code  fipstate  fipscty  BA2005  BA2006  \
0  New Jersey    Atlantic        34001        34        1    2281    2314   
1  New Jersey      Bergen        34003        34        3   11218   11633   
2  New Jersey  Burlington        34005        34        5    3816    3904   
3  New Jersey      Camden        34007        34        7    3953    4322   
4  New Jersey    Cape May        34009        34        9    1080    1002   

   BA2007  BA2008  BA2009  ...  BA2014  BA2015  BA2016  BA2017  BA2018  \
0    2336    2227    2050  ...    2163    2161    2113    2195    2275   
1   11833   11537   11091  ...   11854   12345   12533   13497   14780   
2    4103    3927    3470  ...    3576    3601    3704    4084    4534   
3    4398    4336    3812  ...    3855    3918    4107    4321    4740   
4     962     915     774  ...     803    1014     833     863     906   

   BA2019  BA2020  BA2021  BA2022  BA2023  
0    2501    3295    4082    3862    3885  
1   

Drop or keep

In [41]:
# Keep only rows where "State" is "New Jersey"
df2 = df2[df2["State"] == "New Jersey"]

# Reset the index after filtering
df2 = df2.reset_index(drop=True)

# Display the first few rows to verify
print(df2.head())

        State      County  County Code  fipstate  fipscty  BA2005  BA2006  \
0  New Jersey    Atlantic        34001        34        1    2281    2314   
1  New Jersey      Bergen        34003        34        3   11218   11633   
2  New Jersey  Burlington        34005        34        5    3816    3904   
3  New Jersey      Camden        34007        34        7    3953    4322   
4  New Jersey    Cape May        34009        34        9    1080    1002   

   BA2007  BA2008  BA2009  ...  BA2014  BA2015  BA2016  BA2017  BA2018  \
0    2336    2227    2050  ...    2163    2161    2113    2195    2275   
1   11833   11537   11091  ...   11854   12345   12533   13497   14780   
2    4103    3927    3470  ...    3576    3601    3704    4084    4534   
3    4398    4336    3812  ...    3855    3918    4107    4321    4740   
4     962     915     774  ...     803    1014     833     863     906   

   BA2019  BA2020  BA2021  BA2022  BA2023  
0    2501    3295    4082    3862    3885  
1   

In [42]:
df2.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,...,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022,BA2023
0,New Jersey,Atlantic,34001,34,1,2281,2314,2336,2227,2050,...,2163,2161,2113,2195,2275,2501,3295,4082,3862,3885
1,New Jersey,Bergen,34003,34,3,11218,11633,11833,11537,11091,...,11854,12345,12533,13497,14780,14225,15508,18500,17576,18508
2,New Jersey,Burlington,34005,34,5,3816,3904,4103,3927,3470,...,3576,3601,3704,4084,4534,4732,5915,7617,6977,7703
3,New Jersey,Camden,34007,34,7,3953,4322,4398,4336,3812,...,3855,3918,4107,4321,4740,5131,7073,8994,8030,8225
4,New Jersey,Cape May,34009,34,9,1080,1002,962,915,774,...,803,1014,833,863,906,925,1052,1334,1284,1311


In [43]:
# Drop the columns "County Code", "fipstate", and "fipscty"
df2 = df2.drop(columns=["County Code", "fipstate", "fipscty"], errors="ignore")

# Display the first few rows to verify changes
print(df2.head())

        State      County  BA2005  BA2006  BA2007  BA2008  BA2009  BA2010  \
0  New Jersey    Atlantic    2281    2314    2336    2227    2050    2159   
1  New Jersey      Bergen   11218   11633   11833   11537   11091   11437   
2  New Jersey  Burlington    3816    3904    4103    3927    3470    3515   
3  New Jersey      Camden    3953    4322    4398    4336    3812    3946   
4  New Jersey    Cape May    1080    1002     962     915     774     789   

   BA2011  BA2012  ...  BA2014  BA2015  BA2016  BA2017  BA2018  BA2019  \
0    2095    2117  ...    2163    2161    2113    2195    2275    2501   
1   11706   11650  ...   11854   12345   12533   13497   14780   14225   
2    3618    3510  ...    3576    3601    3704    4084    4534    4732   
3    3893    3895  ...    3855    3918    4107    4321    4740    5131   
4     843     881  ...     803    1014     833     863     906     925   

   BA2020  BA2021  BA2022  BA2023  
0    3295    4082    3862    3885  
1   15508   18500   

In [44]:
df2.head()

,State,County,BA2005,BA2006,BA2007,BA2008,BA2009,BA2010,BA2011,BA2012,...,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022,BA2023
0,New Jersey,Atlantic,2281,2314,2336,2227,2050,2159,2095,2117,...,2163,2161,2113,2195,2275,2501,3295,4082,3862,3885
1,New Jersey,Bergen,11218,11633,11833,11537,11091,11437,11706,11650,...,11854,12345,12533,13497,14780,14225,15508,18500,17576,18508
2,New Jersey,Burlington,3816,3904,4103,3927,3470,3515,3618,3510,...,3576,3601,3704,4084,4534,4732,5915,7617,6977,7703
3,New Jersey,Camden,3953,4322,4398,4336,3812,3946,3893,3895,...,3855,3918,4107,4321,4740,5131,7073,8994,8030,8225
4,New Jersey,Cape May,1080,1002,962,915,774,789,843,881,...,803,1014,833,863,906,925,1052,1334,1284,1311


Collabse data

3. Merge at least 3 datasets (at least 2 merges); eg merge on geography (eg state), time (eg year), or some var (eg occupation)